In [2]:
from agent_arena_no_gathering import *
from environments.evil_wgw_env import EvilWindyGridWorld
from hessian_utils import *
from tabular_methods import QTable
import numpy as np
from hessian import hessian
import matplotlib.pyplot as plt
%matplotlib inline

from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
stochasticities = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.2]
sing_vals = {}

In [ ]:
for stoch in stochasticities:
    print(stoch)
    env = EvilWindyGridWorld(grid_size=(7, 10), visual=True, stochasticity=stoch)
    dataset = OracleQDataset(env)
    dataloader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)
    net = DQNOracleAgent()
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    
    for e in range(2):
        for i, batch in enumerate(dataloader):
            optimizer.zero_grad()
            x, y = batch
            y_ = net(x)
            loss = (y - y_).pow(2).mean()  
            loss.backward()
            optimizer.step()
    
                
    states = []
    correct_q = []
    for i in range(env.w):
        for j in range(env.h):
            env.reset()
            valid, state = env.set_pos((i, j))
            if valid:
                states.append(state)
                correct_q.append(dataset.q[i, j, :])

                
    states = np.stack(states, axis=0)
    correct_q = np.stack(correct_q, axis=0)
    loss = (net(torch.Tensor(states)) - torch.Tensor(correct_q)).pow(2).mean()
    
    hess = hessian(loss, net.parameters()).detach().numpy()
    u, s, v = np.linalg.svd(hess, full_matrices=False)
    sing_vals[stoch] = s

In [ ]:
for s in stochasticities:
    plt.semilogy(sing_vals[s][:200] / sing_vals[s][0], label=str(s))
    
    
plt.legend()

In [10]:
env = EvilWindyGridWorld(grid_size=(7, 10), visual=False, stochasticity=0.05)
visual_env = EvilWindyGridWorld(grid_size=(7, 10), visual=True, stochasticity=0.05)

In [11]:
dataset = OracleQRDataset(env, visual_env)

In [12]:
net = QROracleAgent()

dataloader = DataLoader(dataset=dataset, batch_size=64, shuffle=True)
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [13]:
for e in range(3):
    for i, batch in enumerate(dataloader):
        optimizer.zero_grad()
        x, y = batch
        y_ = net(x)
        loss = (y - y_).pow(2).mean()  
        loss.backward()
        optimizer.step()
        print(loss.item())

0.03754480928182602
0.03532855585217476
0.03475535660982132
0.03290678188204765
0.0319015234708786
0.030598465353250504
0.02953595295548439
0.028919430449604988
0.027243148535490036
0.02650080993771553
0.02556690201163292
0.024749144911766052
0.024132106453180313
0.02327345684170723
0.022836975753307343
0.022022364661097527
0.02135894075036049
0.021139932796359062
0.02050241082906723
0.019519906491041183
0.01890464872121811
0.018735608085989952
0.018279384821653366
0.017616041004657745
0.017519323155283928
0.01712104119360447
0.016713963821530342
0.016270503401756287
0.015784310176968575
0.015559273771941662
0.015012326650321484
0.014726351015269756
0.014359764754772186
0.013801571913063526
0.013699104078114033
0.01339760422706604
0.01282261498272419
0.012481705285608768
0.012367152608931065
0.011767979711294174
0.011388526298105717
0.011344821192324162
0.010983506217598915
0.01059273723512888
0.010455118492245674
0.010041944682598114
0.00973245408385992
0.009509862400591373
0.00932691

0.00014710149844177067
0.0002851250465027988
0.0002528154291212559
0.0002456640941090882
0.00021698823547922075
0.00021303544053807855
0.0003043886390514672
0.00023772247368469834
0.00035332710831426084
0.00019055514712817967
0.0002209208469139412
0.0002158643474103883
0.0003149773692712188
0.00022459565661847591
0.00019498246547300369
0.0002846309798769653
0.0002668257802724838
0.00017051128088496625
0.00025758662377484143
0.00023101124679669738
0.00016007083468139172
0.00023100673570297658
0.00029181185527704656
0.00019034463912248611
0.0002397912321612239
0.0002730857231654227
0.00014627138443756849
0.00025874850689433515
0.00026642202283255756
0.0002602070162538439
0.0002574698009993881
0.00021202339848969132
0.0001994882768485695
0.00024743983522057533
0.0003419644490350038
0.00029806719976477325
0.00016979507927317172
0.00013477637548930943
0.0002493846113793552
0.00027411250630393624
0.00021305448899511248
0.00028627776191569865
0.0002272806450491771
0.00016175169730558991
0.000

0.0001364752824883908
0.00011968769831582904
0.00010709404887165874
9.858411794994026e-05
0.0001703942398307845
9.972743282560259e-05
0.00013135580229572952
0.00013014314754400402
9.83850477496162e-05
0.00010392763215349987
0.0001126280621974729
0.00012181838974356651
7.08073639543727e-05
9.843255975283682e-05
0.00010193973866989836
0.00011169330537086353
9.132741979556158e-05
0.00018428851035423577
0.0001094139734050259
0.00015995497233234346
0.0001271249639103189
0.00015497715503443033
0.00010981370724039152
9.161117486655712e-05
0.00013184724957682192
9.406457684235647e-05
8.21947687654756e-05
0.00017499960085842758
7.627180457348004e-05
8.826130942907184e-05
7.464639202225953e-05
7.37283262424171e-05
0.00011433615873102099
0.0001351676182821393
0.0001226194726768881
0.0001483039086451754
0.0001218600373249501
0.00011454163177404553
5.3574978664983064e-05
9.409221092937514e-05
9.260839578928426e-05
7.679685222683474e-05
0.00010005982767324895
7.509651914006099e-05
0.0001015055458992

5.394888648879714e-05
4.109578367206268e-05
6.431195652112365e-05
3.972838385379873e-05
6.547132943524048e-05
6.261745875235647e-05
4.729751526610926e-05
5.222986510489136e-05
6.200421194080263e-05
4.386858199723065e-05
3.067796933464706e-05
4.9421061703469604e-05
2.822771784849465e-05
4.236814129399136e-05
5.545357271330431e-05
4.1869749111356214e-05
7.390268729068339e-05
3.0993825930636376e-05
5.586411862168461e-05
4.570517921820283e-05
4.270236604497768e-05
3.625608223956078e-05
5.334609159035608e-05
4.387335502542555e-05
5.565103128901683e-05
6.233905151020736e-05
4.162405093666166e-05
3.669521538540721e-05
5.247810622677207e-05
5.3619172831531614e-05
4.6874778490746394e-05
4.949553112965077e-05
5.6584845879115164e-05
3.202550578862429e-05
4.7913963499013335e-05
5.869086453458294e-05
5.2120754844509065e-05
3.7562931538559496e-05
5.3757998102810234e-05
6.366028537740931e-05
2.546263931435533e-05
6.307514559011906e-05
3.8839563785586506e-05
4.848021490033716e-05
3.694097540574148e-05

2.068886169581674e-05
2.8833812393713742e-05
2.859087908291258e-05
2.9000146241742186e-05
2.495066473784391e-05
2.3528024030383676e-05
3.407073381822556e-05
1.7788326658774167e-05
3.845374521915801e-05
1.8654089217307046e-05
3.769393151742406e-05
3.079620000789873e-05
2.5294028091593646e-05
2.965602106996812e-05
3.952125916839577e-05
3.0479819542961195e-05
2.7654315999825485e-05
2.5347990231239237e-05
3.831762296613306e-05
1.8875562091125175e-05
2.782717638183385e-05
4.19571042584721e-05
2.716268863878213e-05
2.512985884095542e-05
2.9873146559111774e-05
4.0442508179694414e-05
3.095554711762816e-05
1.5542824257863685e-05
3.995447332272306e-05
4.5663367927772924e-05
3.541411570040509e-05
2.5762356017366983e-05
1.5693254681536928e-05
3.140997068840079e-05
3.6578159779310226e-05
2.944454172393307e-05
3.099155583186075e-05
4.4770811655325815e-05
2.5090525014093146e-05
4.0601997170597315e-05
2.2332766093313694e-05
2.1574587663053535e-05
3.731430842890404e-05
3.453847239143215e-05
4.646994420

...

2.520409725548234e-05
2.4857170501491055e-05
4.7238987463060766e-05
2.7033238438889384e-05
1.763828913681209e-05
1.60244781000074e-05
1.9068065739702433e-05
2.6374538720119745e-05
3.871185253956355e-05
1.2787440937245265e-05
1.9007486116606742e-05
2.5397119316039607e-05
2.66522492893273e-05
1.927436824189499e-05
2.5695684598758817e-05
2.005682654271368e-05
3.0167049771989696e-05
2.8389215003699064e-05
1.9119663193123415e-05
2.1896432372159325e-05
1.317872647632612e-05
2.545577081036754e-05
2.1161502445465885e-05
3.094911517109722e-05
1.8432574506732635e-05
1.2284624972380698e-05
2.1584091882687062e-05
2.9543372875195928e-05
1.6160898667294532e-05
1.5606874512741342e-05
2.519498229958117e-05
2.968761691590771e-05
2.436096838209778e-05
4.037321559735574e-05
4.510166036197916e-05
1.7877267964649945e-05
2.914731521741487e-05
2.193881300627254e-05
2.6458488719072193e-05
1.6652527847327292e-05
2.6456335035618395e-05
2.9102306143613532e-05
1.4892026229063049e-05
1.8384143913863227e-05
1.91276

1.828713357099332e-05
2.128685991920065e-05
9.569356734573375e-06
2.200073686253745e-05
2.7996480639558285e-05
9.297610631620046e-06
1.2707168934866786e-05
1.6973213860183023e-05
2.476831286912784e-05
2.415902963548433e-05
2.8040407414664514e-05
1.8183905922342092e-05
1.430307293048827e-05
2.718740506679751e-05
1.3394194866123144e-05
2.7102189051220194e-05
1.2397679711284582e-05
1.822236845328007e-05
3.116604420938529e-05
9.366716767544858e-06
1.3770566511084326e-05
1.766682180459611e-05
2.251948171760887e-05
2.2130996512714773e-05
2.4509943614248186e-05
3.2681105949450284e-05
2.401680831098929e-05
9.194841368298512e-06
1.6693818906787783e-05
1.2794356734957546e-05
1.6571662854403257e-05
2.076898090308532e-05
1.2187693755549844e-05
9.927427527145483e-06
1.7750136976246722e-05
2.4137501895893365e-05
1.2153210263932124e-05
1.6000323739717714e-05
2.5190425731125288e-05
2.250949910376221e-05
1.874329063866753e-05
1.5014614291430917e-05
2.9050304874544963e-05
2.1354930140660144e-05
2.561422

2.0724241039715707e-05
3.224525062250905e-05
2.0257379219401628e-05
1.6126285117934458e-05
2.4178336389013566e-05
1.7915659555001184e-05
2.3584076188853942e-05
1.276366674574092e-05
1.2537779184640385e-05
1.3313223462319002e-05
2.1198888134676963e-05
1.4517243471345864e-05
2.650920214364305e-05
1.1658509720291477e-05
2.5594368707970716e-05
2.219173802586738e-05
2.2016640286892653e-05
1.2516888091340661e-05
1.5709500075899996e-05
1.8720525986282155e-05
1.7419864889234304e-05
1.6623474948573858e-05
1.4557075701304711e-05
2.2661130060441792e-05
2.1880805434193462e-05
1.1470340723462868e-05
1.2092921679141e-05
2.9387836548266932e-05
3.344129800098017e-05
1.535009141662158e-05
2.0363006115076132e-05
1.9754848835873418e-05
1.6561518350499682e-05
1.3797760402667336e-05
1.3962989214633126e-05
1.6604904885753058e-05
1.894709566840902e-05
1.979004082386382e-05
1.8233036826131865e-05
1.2800715921912342e-05
1.1703750715241767e-05
2.0961575501132756e-05
2.780906652333215e-05
9.439573659619782e-06
1

5.797231096948963e-06
1.2222988516441546e-05
1.0783662219182588e-05
4.00640674342867e-05
1.7720722098601982e-05
9.949875675374642e-06
1.3546626178140286e-05
1.2795144357369281e-05
2.1746522179455496e-05
1.870987398433499e-05
2.4282369849970564e-05
1.148283263319172e-05
2.2064528820919804e-05
1.6445286746602505e-05
9.833956937654875e-06
1.9236158550484106e-05
1.3942515579401515e-05
8.962264473666437e-06
2.3428376152878627e-05
1.651959610171616e-05
1.559147312946152e-05
9.549192327540368e-06
1.5685682228649966e-05
1.5005098248366266e-05
1.1657763934636023e-05
9.175166269415058e-06
1.0381183528807014e-05
1.2338052329141647e-05
1.985329799936153e-05
1.9556931874831207e-05
2.067731475108303e-05
8.587138836446684e-06
1.21598668556544e-05
1.0195644790655933e-05
1.161577893071808e-05
1.8717797502176836e-05
1.4108486539043952e-05
2.8202075554872863e-05
1.785897256922908e-05
1.789603993529454e-05
7.3438013714621775e-06
1.0468842447153293e-05
2.02884002646897e-05
8.319387234223541e-06
1.100412555

1.123605397879146e-05
7.429419383697677e-06
9.8086275102105e-06
1.4862789612379856e-05
1.0765384104161058e-05
1.8877712136600167e-05
1.1213805919396691e-05
2.0774452423211187e-05
4.133131369599141e-06
1.6423915440100245e-05
1.3565124390879646e-05
1.070162579708267e-05
1.7011780073517002e-05
1.0506744729354978e-05
1.8546257706475444e-05
7.702533366682474e-06
1.722695742500946e-05
1.6716043319320306e-05
7.9286728578154e-06
9.911158485920168e-06
1.7310552721028216e-05
1.1719301255652681e-05
1.1103876204288099e-05
1.1572104995138943e-05
1.419295494997641e-05
8.86603447725065e-06
8.980281563708559e-06
2.398187280050479e-05
1.390044781146571e-05
1.2911557860206813e-05
1.613436506886501e-05
1.0609160199237522e-05
9.16665158001706e-06
1.561046883580275e-05
9.374087312608026e-06
8.798097951512318e-06
1.301450356550049e-05
7.863794053264428e-06
1.667169271968305e-05
1.2222197256051004e-05
1.6566951671848074e-05
9.291716196457855e-06
2.0255432900739834e-05
1.8333070329390466e-05
6.30078102403786e

In [17]:
states = []
correct_z = []
for i in range(env.w):
    for j in range(env.h):
        visual_env.reset()
        valid, state = visual_env.set_pos((i, j))
        if valid:
            states.append(state)
            correct_z.append(dataset.z[i, j, :, :])
